In [58]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

In [65]:
rb = pd.read_csv('../input/rb/rb_stats.csv')

In [66]:
rb.shape

(6907, 19)

In [67]:
rb.sort_values(['year','Week'],inplace=True)
rb.head()

,year,Name,Week,Opponent,games,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma,att_car,yds_car,td_car,tar_car,rec_yds_car,surface_0,h/a_1,def_rk,points
21,2015,Aaron Ripkowski,1,CHI,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,22,0.0
68,2015,Adrian Peterson,1,SF,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,32,8.2
225,2015,Alfred Blue,1,KC,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,5.9
276,2015,Alfred Morris,1,MIA,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,31,15.1
361,2015,Ameer Abdullah,1,LAC,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,26,19.4


In [68]:
rb.reset_index(inplace =True, drop=True)

In [69]:
def conv_atts(df):
    df.rename(columns={'Unnamed: 0':'player'}, inplace=True)
    df.replace('null',0, inplace=True)
    df['h/a_1'] = df['h/a_1'].astype('category')
    df['surface_0'] = df['surface_0'].astype('category')
    

    return df

In [70]:
rb = conv_atts(rb)
rb.dtypes


year              int64
Name             object
Week              int64
Opponent         object
games             int64
att_ma          float64
yds_ma          float64
td_ma           float64
tar_ma          float64
rec_yds_ma      float64
att_car         float64
yds_car         float64
td_car          float64
tar_car         float64
rec_yds_car     float64
surface_0      category
h/a_1          category
def_rk            int64
points          float64
dtype: object

In [71]:

rb.isnull().sum()

year           0
Name           0
Week           0
Opponent       0
games          0
att_ma         0
yds_ma         0
td_ma          0
tar_ma         0
rec_yds_ma     0
att_car        0
yds_car        0
td_car         0
tar_car        0
rec_yds_car    0
surface_0      0
h/a_1          0
def_rk         0
points         0
dtype: int64

In [72]:
test = rb[rb['year']==2018]
test=test[test['Week']==8]
test.shape

(95, 19)

In [73]:
test.index

Int64Index([6812, 6813, 6814, 6815, 6816, 6817, 6818, 6819, 6820, 6821, 6822,
            6823, 6824, 6825, 6826, 6827, 6828, 6829, 6830, 6831, 6832, 6833,
            6834, 6835, 6836, 6837, 6838, 6839, 6840, 6841, 6842, 6843, 6844,
            6845, 6846, 6847, 6848, 6849, 6850, 6851, 6852, 6853, 6854, 6855,
            6856, 6857, 6858, 6859, 6860, 6861, 6862, 6863, 6864, 6865, 6866,
            6867, 6868, 6869, 6870, 6871, 6872, 6873, 6874, 6875, 6876, 6877,
            6878, 6879, 6880, 6881, 6882, 6883, 6884, 6885, 6886, 6887, 6888,
            6889, 6890, 6891, 6892, 6893, 6894, 6895, 6896, 6897, 6898, 6899,
            6900, 6901, 6902, 6903, 6904, 6905, 6906],
           dtype='int64')

In [74]:
# Design train and test datasets

train = rb[:test.index[0]]
train.tail()

,year,Name,Week,Opponent,games,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma,att_car,yds_car,td_car,tar_car,rec_yds_car,surface_0,h/a_1,def_rk,points
6807,2018,Trenton Cannon,7,MIN,9,0.75,1.875,0.00,0.125000,0.750000,0.75,1.88,0.00,0.12,0.75,0.0,1.0,6,11.3
6808,2018,Tyler Ervin,7,JAX,25,0.00,0.000,0.00,0.250000,1.500000,0.00,0.00,0.00,0.22,1.33,1.0,0.0,10,2.9
6809,2018,Wayne Gallman,7,ATL,22,2.75,10.000,0.00,2.000000,9.875000,4.11,18.78,0.00,2.00,8.78,0.0,0.0,30,0.0
6810,2018,Wendell Smallwood,7,CAR,30,7.25,37.000,0.25,3.375000,18.250000,6.89,33.56,0.22,3.33,18.89,1.0,1.0,7,5.7
6811,2018,Zach Line,7,BAL,51,0.00,0.000,0.00,1.428571,2.285714,0.12,0.62,0.00,1.38,2.38,1.0,0.0,1,0.0


In [75]:
train.shape

(6812, 19)

In [76]:
test.columns

Index(['year', 'Name', 'Week', 'Opponent', 'games', 'att_ma', 'yds_ma',
       'td_ma', 'tar_ma', 'rec_yds_ma', 'att_car', 'yds_car', 'td_car',
       'tar_car', 'rec_yds_car', 'surface_0', 'h/a_1', 'def_rk', 'points'],
      dtype='object')

In [77]:
test[test['Name']=='Saquon Barkley']

,year,Name,Week,Opponent,games,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma,att_car,yds_car,td_car,tar_car,rec_yds_car,surface_0,h/a_1,def_rk,points
6891,2018,Saquon Barkley,8,WAS,10,13.888889,67.444444,0.777778,8.222222,57.222222,13.89,67.44,0.78,8.22,57.22,0.0,1.0,7,20.1


In [78]:
# Target variables 

y_train = train['points']
y_test = test['points']

# Predictors, all numeric variables minus target variable
X_train = train.iloc[:,4:-1]
X_test = test.iloc[:,4:-1]

In [79]:
scaler = MinMaxScaler()
scaler.fit_transform(X_train)

array([[0.        , 0.        , 0.01568627, ..., 1.        , 0.        ,
        0.67741935],
       [0.        , 0.        , 0.01568627, ..., 1.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.01568627, ..., 0.        , 1.        ,
        0.06451613],
       ...,
       [0.375     , 0.1125    , 0.09411765, ..., 0.        , 0.        ,
        0.93548387],
       [0.51785714, 0.29659091, 0.30588235, ..., 1.        , 1.        ,
        0.19354839],
       [0.89285714, 0.        , 0.01568627, ..., 1.        , 0.        ,
        0.        ]])

In [80]:
X_test.shape

(95, 14)

In [81]:
X_train.shape

(6812, 14)

In [82]:
lr = LinearRegression()

In [83]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [84]:
coeff = pd.DataFrame(X_train.columns)
coeff['coefficients'] = lr.coef_
coeff



,0,coefficients
0,games,-0.020439
1,att_ma,-0.177702
2,yds_ma,0.046018
3,td_ma,1.292675
4,tar_ma,0.212350
5,rec_yds_ma,0.003375
6,att_car,0.316650
7,yds_car,0.040105
8,td_car,1.299873
9,tar_car,0.730154


In [85]:
X_train.shape

(6812, 14)

In [86]:
yhat = lr.predict(X_test)

In [87]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse_val = rmse(yhat, y_test)
print("RMSE error is: " + str(rmse_val))

RMSE error is: 6.015637215300407


In [88]:
results = pd.DataFrame()
results['Player'] = test['Name']
#results['score'] = y_test
results['Predicted'] = yhat
results['points'] = test['points']
#results['Week'] = qb[qb['year']==2017]['Week']
#results.groupby('Player').mean()[['score','Predicted']]
results

,Player,Predicted,points
6812,Aaron Jones,7.932638,16.6
6813,Adrian Peterson,15.836242,30.6
6814,Alan Cross,0.602536,0.0
6815,Alex Armah,1.229035,0.0
6816,Alex Collins,11.408434,13.3
6817,Alfred Blue,8.667361,4.2
6818,Alfred Morris,5.101730,2.8
6819,Alvin Kamara,27.136487,26.6
6820,Ameer Abdullah,3.127331,0.6
6821,Andy Janovich,1.664597,0.0


In [323]:
results.

(300, 3)

In [26]:
results.to_csv('../output/rb_test.csv', index= False)